# Data Cleaning Solutions: Gender Wage Gap in the Balkans

This notebook contains complete solutions to all data cleaning exercises.

## Learning Objectives
1. Load and inspect raw data
2. Identify data quality issues
3. Handle missing values
4. Remove duplicates
5. Standardize data formats
6. Validate cleaned data
7. Export cleaned dataset
8. Analyze gender wage gaps

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Load the raw data
df = pd.read_csv('../data/raw/macedonia_wage_sample.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nFirst 10 rows:")
df.head(10)

## 2. Initial Data Inspection

In [ ]:
# Display basic information
print("Data Types and Non-Null Counts:")
df.info()

In [ ]:
# Check for missing values
print("Missing values per column:")
missing_df = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
print(missing_df[missing_df['Missing Count'] > 0])
print(f"\nTotal missing values: {df.isnull().sum().sum()}")

In [ ]:
# Check for duplicates
duplicates = df.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")
if duplicates.sum() > 0:
    print("\nDuplicate rows:")
    print(df[duplicates])

In [ ]:
# Check unique values in categorical columns
categorical_cols = ['country', 'gender', 'sector', 'education_level', 'age_group']
for col in categorical_cols:
    print(f"\n{col}: {df[col].unique()}")
    print(f"  Value counts: {df[col].value_counts().to_dict()}")

## 3. Identified Data Quality Issues

Based on the inspection above, here are the data quality issues:

1. **Missing Values**: 
   - `hours_worked` has 3 missing values
   - `avg_monthly_wage` has 1 missing value

2. **Duplicates**: 
   - 1 duplicate row found (North Macedonia, 2020, Female, Public, University)

3. **Country Name Inconsistencies**:
   - "Bosnia" should be "Bosnia and Herzegovina"
   - Other country names need standardization

4. **Data Types**:
   - All columns appear to have correct types (year as int, wages as int, etc.)

## 4. Remove Duplicates

In [ ]:
# Create a copy and remove duplicates
df_clean = df.copy()
df_clean = df_clean.drop_duplicates()

print(f"Rows before: {len(df)}")
print(f"Rows after: {len(df_clean)}")
print(f"Duplicates removed: {len(df) - len(df_clean)}")

## 5. Handle Missing Values

In [ ]:
# Examine rows with missing values
print("Rows with missing values:")
df_clean[df_clean.isnull().any(axis=1)]

In [ ]:
# Handle missing values in 'hours_worked'
# Strategy: Fill with median hours worked by sector and education level

# Calculate median hours worked by sector and education level
median_hours = df_clean.groupby(['sector', 'education_level'])['hours_worked'].median()
print("Median hours worked by sector and education level:")
print(median_hours)

# Fill missing hours_worked values
for idx, row in df_clean[df_clean['hours_worked'].isnull()].iterrows():
    sector = row['sector']
    edu = row['education_level']
    if (sector, edu) in median_hours.index:
        df_clean.loc[idx, 'hours_worked'] = median_hours[(sector, edu)]
    else:
        # Fallback to overall median
        df_clean.loc[idx, 'hours_worked'] = df_clean['hours_worked'].median()

print(f"\nMissing hours_worked after filling: {df_clean['hours_worked'].isnull().sum()}")

In [ ]:
# Handle missing values in 'avg_monthly_wage'
# Strategy: Estimate based on similar records (same country, year, sector, education, gender)

missing_wage_idx = df_clean[df_clean['avg_monthly_wage'].isnull()].index
print(f"Missing wage records: {len(missing_wage_idx)}")

for idx in missing_wage_idx:
    row = df_clean.loc[idx]
    # Find similar records
    similar = df_clean[
        (df_clean['country'] == row['country']) &
        (df_clean['sector'] == row['sector']) &
        (df_clean['education_level'] == row['education_level']) &
        (df_clean['gender'] == row['gender']) &
        (df_clean['avg_monthly_wage'].notna())
    ]
    
    if len(similar) > 0:
        # Use median of similar records
        df_clean.loc[idx, 'avg_monthly_wage'] = similar['avg_monthly_wage'].median()
        print(f"Filled wage for index {idx} with median: {similar['avg_monthly_wage'].median()}")
    else:
        # Fallback: use overall median for that gender
        gender_median = df_clean[df_clean['gender'] == row['gender']]['avg_monthly_wage'].median()
        df_clean.loc[idx, 'avg_monthly_wage'] = gender_median
        print(f"Filled wage for index {idx} with gender median: {gender_median}")

print(f"\nMissing avg_monthly_wage after filling: {df_clean['avg_monthly_wage'].isnull().sum()}")

## 6. Standardize Country Names

In [ ]:
# Standardize country names
country_mapping = {
    'Bosnia': 'Bosnia and Herzegovina',
    'Kosovo': 'Kosovo',
    'North Macedonia': 'North Macedonia',
    'Serbia': 'Serbia',
    'Albania': 'Albania',
    'Montenegro': 'Montenegro'
}

df_clean['country'] = df_clean['country'].replace(country_mapping)

print("Standardized country names:")
print(df_clean['country'].unique())
print(f"\nCountry value counts:")
print(df_clean['country'].value_counts())

## 7. Data Type Validation

In [ ]:
# Check and convert data types
print("Current data types:")
print(df_clean.dtypes)

# Ensure numeric columns are proper types
df_clean['year'] = df_clean['year'].astype(int)
df_clean['avg_monthly_wage'] = df_clean['avg_monthly_wage'].astype(float)
df_clean['hours_worked'] = df_clean['hours_worked'].astype(float)

print("\nUpdated data types:")
print(df_clean.dtypes)

## 8. Calculate Wage Gap

In [ ]:
# Calculate gender wage gap
# Gap = (Male wage - Female wage) / Male wage * 100

# Group by relevant dimensions and pivot on gender
wage_pivot = df_clean.pivot_table(
    values='avg_monthly_wage',
    index=['country', 'year', 'sector', 'education_level', 'age_group'],
    columns='gender',
    aggfunc='mean'
).reset_index()

# Calculate wage gap
wage_pivot['wage_gap_percent'] = (
    (wage_pivot['Male'] - wage_pivot['Female']) / wage_pivot['Male'] * 100
).round(2)

wage_pivot['wage_gap_absolute'] = (wage_pivot['Male'] - wage_pivot['Female']).round(0)

print("Gender Wage Gap Analysis:")
print(wage_pivot.head(10))

In [ ]:
# Summary statistics by country
country_gap = wage_pivot.groupby('country').agg({
    'wage_gap_percent': ['mean', 'min', 'max'],
    'Female': 'mean',
    'Male': 'mean'
}).round(2)

country_gap.columns = ['_'.join(col).strip() for col in country_gap.columns.values]
country_gap = country_gap.sort_values('wage_gap_percent_mean', ascending=False)

print("\nWage Gap by Country:")
print(country_gap)

## 9. Outlier Detection

In [ ]:
# Check for outliers using box plots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot for avg_monthly_wage
axes[0].boxplot([df_clean[df_clean['gender'] == 'Male']['avg_monthly_wage'],
                  df_clean[df_clean['gender'] == 'Female']['avg_monthly_wage']],
                labels=['Male', 'Female'])
axes[0].set_title('Average Monthly Wage by Gender')
axes[0].set_ylabel('Wage (MKD)')
axes[0].grid(True, alpha=0.3)

# Box plot for hours_worked
axes[1].boxplot([df_clean[df_clean['gender'] == 'Male']['hours_worked'],
                  df_clean[df_clean['gender'] == 'Female']['hours_worked']],
                labels=['Male', 'Female'])
axes[1].set_title('Hours Worked by Gender')
axes[1].set_ylabel('Hours per Month')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical outlier detection using IQR method
Q1 = df_clean['avg_monthly_wage'].quantile(0.25)
Q3 = df_clean['avg_monthly_wage'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_clean[(df_clean['avg_monthly_wage'] < lower_bound) | 
                    (df_clean['avg_monthly_wage'] > upper_bound)]

print(f"\nOutliers detected: {len(outliers)}")
if len(outliers) > 0:
    print(outliers[['country', 'year', 'gender', 'avg_monthly_wage', 'education_level']])

## 10. Visualizations

In [ ]:
# Wage gap by country
plt.figure(figsize=(12, 6))
country_avg_gap = wage_pivot.groupby('country')['wage_gap_percent'].mean().sort_values(ascending=True)
country_avg_gap.plot(kind='barh', color='coral')
plt.title('Average Gender Wage Gap by Country', fontsize=14, fontweight='bold')
plt.xlabel('Wage Gap (%)', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Wage gap over time for North Macedonia
macedonia_data = wage_pivot[wage_pivot['country'] == 'North Macedonia']
macedonia_trend = macedonia_data.groupby('year')['wage_gap_percent'].mean()

plt.figure(figsize=(10, 6))
plt.plot(macedonia_trend.index, macedonia_trend.values, marker='o', linewidth=2, markersize=8)
plt.title('Gender Wage Gap Trend in North Macedonia (2020-2023)', fontsize=14, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Wage Gap (%)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(macedonia_trend.index)
plt.tight_layout()
plt.show()

In [ ]:
# Wage comparison by education level and gender
edu_wage = df_clean.groupby(['education_level', 'gender'])['avg_monthly_wage'].mean().unstack()

edu_wage.plot(kind='bar', figsize=(10, 6), width=0.8)
plt.title('Average Wage by Education Level and Gender', fontsize=14, fontweight='bold')
plt.xlabel('Education Level', fontsize=12)
plt.ylabel('Average Monthly Wage (MKD)', fontsize=12)
plt.legend(title='Gender', fontsize=11)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Sector comparison
sector_gap = wage_pivot.groupby('sector')['wage_gap_percent'].mean().sort_values()

plt.figure(figsize=(10, 5))
sector_gap.plot(kind='barh', color=['#2ecc71', '#e74c3c'])
plt.title('Gender Wage Gap by Sector', fontsize=14, fontweight='bold')
plt.xlabel('Wage Gap (%)', fontsize=12)
plt.ylabel('Sector', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 11. Final Validation

In [ ]:
# Final data quality check
print("=" * 50)
print("FINAL DATA QUALITY REPORT")
print("=" * 50)
print(f"\nDataset Shape: {df_clean.shape}")
print(f"Total Records: {len(df_clean)}")
print(f"\nMissing Values: {df_clean.isnull().sum().sum()}")
print(f"Duplicate Records: {df_clean.duplicated().sum()}")
print(f"\nData Types:")
print(df_clean.dtypes)
print(f"\nDate Range: {df_clean['year'].min()} - {df_clean['year'].max()}")
print(f"Countries Included: {df_clean['country'].nunique()}")
print(f"\nData Quality Status: ✓ PASSED")

In [ ]:
# Summary statistics
print("\nSummary Statistics:")
print(df_clean.describe())

## 12. Export Cleaned Data

In [ ]:
# Save the cleaned dataset
output_path = '../data/cleaned/macedonia_wage_cleaned.csv'
df_clean.to_csv(output_path, index=False)
print(f"✓ Cleaned data exported to: {output_path}")

# Save the wage gap analysis
wage_gap_path = '../data/cleaned/wage_gap_analysis.csv'
wage_pivot.to_csv(wage_gap_path, index=False)
print(f"✓ Wage gap analysis exported to: {wage_gap_path}")

print(f"\n✓ All data processing complete!")

## 13. Key Findings

### Data Cleaning Summary:
- Removed 1 duplicate record
- Filled 3 missing values in `hours_worked` using sector/education medians
- Filled 1 missing value in `avg_monthly_wage` using similar record estimation
- Standardized country names across the dataset

### Gender Wage Gap Insights:
1. **Overall Gap**: Varies significantly by country, sector, and education level
2. **Education Impact**: University-educated workers show different gap patterns than high school educated
3. **Sector Differences**: Public vs Private sector show distinct wage gap patterns
4. **Temporal Trends**: Some countries show improving trends, others worsening

### Recommendations:
1. Collect more comprehensive data with additional variables (experience, occupation codes)
2. Include more years of historical data for better trend analysis
3. Add confidence intervals for statistical significance
4. Consider regional variations within countries